In [2]:
import pandas as pd
import os

In [8]:
filepath = r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata'
for root,dirs,files in os.walk(filepath):
    for file in files:
        print(os.path.join(root,file))
        print(os.path.join(file[:-4]))

I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\lv1_08.csv
lv1_08
I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\lv1_20.csv
lv1_20
I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\Relative_humidity_08.csv
Relative_humidity_08
I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\Relative_humidity_20.csv
Relative_humidity_20
I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\statistics.csv
statistics
I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\Temperature_08.csv
Temperature_08
I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\Temperature_20.csv
Temperature_20
